# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [26]:
# imports
import requests
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import ollama
from dotenv import load_dotenv
import os

In [17]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"

In [18]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.strip() == api_key and api_key.startswith("sk-proj"):
    print("API key retrieved successfully!")
else:
    print("Error retrieving API key")

openai = OpenAI()


API key retrieved successfully!


In [19]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [27]:
# Get gpt-4o-mini to answer, with streaming
messages = [{"role": "user", "content": question}]
stream = openai.chat.completions.create(stream=True, model="gpt-4o-mini", messages=messages)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)


The code snippet you've provided makes use of Python's generator feature along with a set comprehension. Here's a breakdown of what it does:

### Explanation

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` creates a set of authors from a collection called `books`.
   - `book.get("author")` retrieves the value associated with the key `"author"` for each `book` in `books`.
   - The `if book.get("author")` part is a filter that ensures only books with a valid (non-None, non-empty) author are included in the set.
   - The result is a set that contains unique author names. If multiple books have the same author, that author will only appear once in the set.

2. **Yield from**:
   - `yield from` is used to yield values from an iterable. In this case, it yields each element from the set generated by the set comprehension one at a time.
   - This means that if you iterate over the function that contains this line, it will return each unique author name as a separate value.

### Why Use This Code

- **Efficiency**: Sets automatically handle duplicates. Thus, by using a set comprehension, you ensure that each author is represented only once, leveraging the properties of the set data structure.
- **Conciseness**: The `yield from` construct allows you to easily yield all values from an iterable without needing to write additional loops.
- **Readable Intent**: The code communicates its intent clearly—it's creating a unique list of authors from a collection of books and yielding them one by one.

### Example

If `books` is defined as follows:

python
books = [
    {"author": "Author A"},
    {"author": "Author B"},
    {"author": "Author A"},  # Duplicate author
    {"author": None},        # No author
    {"author": "Author C"},
]


Using the provided code, what you would yield would be a set of unique authors: `{"Author A", "Author B", "Author C"}`, and when iterated over, it would yield each author name individually. 

In summary, the code efficiently gathers unique author names from a collection of books and allows iteration over these names through the `yield from` syntax.

In [9]:
# Get Llama 3.2 to answer
messages = [{"role": "user", "content": question}]
payload = {
    "model": MODEL_LLAMA,
    "messages": messages,
    "stream": False
}
response = requests.post(url=OLLAMA_API, json=payload)

In [14]:
print(response.json()["message"]["content"])

This is a Python expression that uses the `yield from` syntax to generate an iterable sequence of author names from a list of book dictionaries.

Here's a breakdown:

* `for book in books`: This part iterates over each dictionary in the `books` list.
* `.get("author")`: For each book, this tries to retrieve its "author" value. If the key does not exist (i.e., the book doesn't have an author), it returns `None`.
* `{...}`: This is a generator expression. It takes the results of the previous steps and converts them into a sequence of values.
* `yield from ...`: This syntax tells Python to delegate the iteration over the sequence generated by the inner expression (`{...}`) to this outer scope.

So, when you put it all together, the expression yields each author name as soon as their corresponding book is processed. The resulting iterable contains author names from books that have an "author" key in their dictionary.

Here's a more readable version of the code:
```python
for author in (boo